In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
import pandas as pd
import networkx as nx
import pylab
import random
import numpy as np
import matplotlib.pyplot as plt
from itertools import groupby
from scipy.stats import kendalltau
from IPython.display import display, HTML
from tabulate import tabulate

In [ ]:
# path of datasets

KARATE = '/content/drive/MyDrive/Complex Networks Project/karate.txt'
JAZZ = '/content/drive/MyDrive/Complex Networks Project/Jazz.txt'
POLBOOKS = '/content/drive/MyDrive/Complex Networks Project/polbooks.txt'

------
<h1>GLI<h1>


In [ ]:
# Change the dataset here
path = KARATE


def simjkd(u, v):
    set_v = set(G.neighbors(v))
    set_v.add(v)
    set_u = set(G.neighbors(u))
    set_u.add(u)
    jac = len(set_v & set_u) * 1.0 / len(set_v | set_u)
    return jac

# Jaccard similarity coefficient
#    jac = (len(set_v & set_u) - 2) * 1.0 / (len(set_v | set_u) - 2)  # Neighborhood redundancy
#    jac = 1 / (1 + abs(len(set_v) - len(set_u)))  # Euclidean distance - measuring similarity
#    jac = len(set_v) * len(set_u) / (pow(len(set_v), 2) + pow(len(set_u), 2) - len(set_v) * len(set_u))
#    Generalized Jaccard similarity


G = nx.Graph()

with open(path) as file:
    for line in file:
        head, tail = [int(x) for x in line.split()]
        G.add_edge(head, tail)

nums = G.number_of_nodes()  # Number of nodes
print('Total number of nodes', nums)

G.remove_edges_from(nx.selfloop_edges(G))
k_shell = nx.core_number(G)
print('k_shell:', k_shell)

maxKshell = max(k_shell.values())  # Using max() to find the maximum value in the k_shell dictionary
minKshell = min(k_shell.values())  # Using min() to find the minimum value in the k_shell dictionary
maxD = max(dict(G.degree()).values())
print("maxD", maxD)

def getCountKshell(G):  # Parameter G represents the graph
    node = G.nodes()
    print("node=", node)
    ks_classfity = [dict(g) for k, g in groupby(sorted(nx.core_number(G).items(), key=by_value), by_value)]
    print(ks_classfity)
    dicts = {}
    for index in node:
        list = []
        print("index=", index)
        for ks_value in ks_classfity:
            dictss = {}
            for k, v in ks_value.items():
                if k == index:
                    continue
                print("k=,length=", k, nx.shortest_path_length(G, source=index, target=k))
                dictss[k] = nx.shortest_path_length(G, source=index, target=k)
            list.append(dictss)
        dicts[index] = list
    return dicts


d = []

# Accumulated sum of influence factor * (k-shell value + degree) + self-degree
res = {}
for nodev in G.nodes():
    value = 0

    for nodeu in G.neighbors(nodev):
        xs = simjkd(nodev, nodeu)
        value += xs * (G.degree(nodeu)) + k_shell[nodeu]
    res[nodev] = value / maxD + G.degree(nodev) + k_shell[nodev]

print(res)

for key in res.keys():
    rest = res[key]
    d.append((key, rest))

sortNum = sorted(d, key=lambda x: x[1], reverse=True)

In [ ]:
# Sort by value
nodelist = []
GLI_sortNum = sorted(res.items(), key=lambda x: x[1], reverse=True)

# Save the result to a file
for key in GLI_sortNum:
    nodelist.append(key.__getitem__(0))
print(nodelist)

output_path = '/content/drive/MyDrive/Complex Networks Project/output_value'
with open(output_path, "w") as f:
    for key, val in sortNum:
        f.write(str(key) + '\t' + str(val) + "\n")

print(f"File saved to {output_path}")

In [ ]:
#We get the 15 most influencial nodes given by the GLI algorithm

GLI_ranking = []
for key in sortNum[:15]:
  GLI_ranking.append(key.__getitem__(0))
print(GLI_ranking)

In [ ]:
# Sort by node ID
nodelist1 = []
sortNum1 = sorted(res.items(), key=lambda x: x[0], reverse=False)

# Save the result to a file
for key in sortNum1:
    nodelist1.append(key.__getitem__(0))
print(nodelist1)

output_path = '/content/drive/MyDrive/Complex Networks Project/output_nodeID'
with open(output_path, "w") as f:
    for key, val in sortNum:
        f.write(str(key) + '\t' + str(val) + "\n")

print(f"File saved to {output_path}")

In [ ]:
#Different Models

DC = nx.degree_centrality(G) #Degree centrality for local information
BC = nx.betweenness_centrality(G) #Betweness centrality Global information
k_shell = nx.core_number(G) # Global information

In [ ]:
DC_sortNum = sorted(DC.items(), key=lambda x: x[1], reverse=True)
BC_sortNum = sorted(BC.items(), key=lambda x: x[1], reverse=True)
kshell_sortNum = sorted(k_shell.items(), key=lambda x: x[1], reverse=True)

In [ ]:
# SIR Model parameters

alpha = 0.02  # Infection probability
recovery_probability = 1.0  # Recovery probability
iterations = 1000  # Number of iterations per node

def simulate_sir(graph, seed_node, alpha):

    """
    Simulate the SIR model for a given seed node.
    Returns the total number of infected nodes (F(t)) for one iteration.
    """

    # Initialize states
    states = {node: "S" for node in graph.nodes()}  # All nodes are initially susceptible
    states[seed_node] = "I"  # Seed node is infected

    infected_count = 1  # Start with the seed node infected
    infected_nodes = [seed_node]

    while infected_nodes:
        new_infections = []
        # Spread the infection
        for node in infected_nodes:
            for neighbor in graph.neighbors(node):
                if states[neighbor] == "S" and random.random() < alpha:
                    states[neighbor] = "I"
                    new_infections.append(neighbor)
            # Node recovers
            states[node] = "R"

        infected_nodes = new_infections  # Update infected nodes
        infected_count += len(new_infections)

    return infected_count

def SIR_ranked(G, alpha, iterations):

  # Calculate average influence (F(t)) over 1000 iterations for each node
  influence_scores = {}
  for node in G.nodes():
      total_influence = 0
      for _ in range(iterations):
          total_influence += simulate_sir(G, node, alpha)
      influence_scores[node] = total_influence / iterations  # Average influence
  # Rank nodes based on their average influence
  return sorted(influence_scores.items(), key=lambda x: x[1], reverse=True)

Ranked_nodes = SIR_ranked(G, alpha, iterations)
for i, key in enumerate(Ranked_nodes):
  if i <= 15:
    print(f"Node {key[0]} | influence:  {key[1]}")
  else: break


In [ ]:
# Creating a DataFrame for comparison
df_comparison = pd.DataFrame({
    'DC': [key[0] for key in DC_sortNum[:15]],
    'BC': [key[0] for key in BC_sortNum[:15]],
    'k-shell': [key[0] for key in kshell_sortNum[:15]],
    'GLI': GLI_ranking[:15],
    'SIR': [key[0] for key in Ranked_nodes[:15]]
})

#change the name of the file as you modified the path
df_comparison.to_csv(f"/content/drive/MyDrive/Complex Networks Project/Comparison table for {name}", index=False)

# Display the comparison table
title = '<h2>Comparison Table for the network</h2>'
display(HTML(title))
print(tabulate(df_comparison, headers='keys', tablefmt='pretty'))

In [ ]:
name_str = ""
if path == KARATE:
    name_str = "Karate"
elif path == JAZZ:
    name_str = "Jazz"
elif path == POLBOOKS:
    name_str = "Polbooks"


rankings = {
    'DC': [key[0] for key in DC_sortNum[:15]],
    'BC': [key[0] for key in BC_sortNum[:15]],
    'k-shell': [key[0] for key in kshell_sortNum[:15]],
    'GLI': GLI_ranking[:15],
}

# Initialize list to store results
results = []

# Simulate SIR and calculate Kendall's tau for each alpha and ranking
alphas = np.arange(0.00, 0.12, 0.02)
for alpha in alphas:
    sir_ranking = [key[0] for key in SIR_ranked(G, alpha, iterations)[:15]]  # Replace SIR_ranked
    for name, ranking in rankings.items():
        tau = kendalltau(sir_ranking, ranking)
        results.append({"alpha": alpha, "method": name, "kendall_tau": tau[0]})

# Create DataFrame
df = pd.DataFrame(results)

# Plot the results
plt.figure(figsize=(10, 6))
for method in rankings.keys():
    subset = df[df["method"] == method]
    plt.plot(subset["alpha"], subset["kendall_tau"], marker='o', label=method)

# Customize the plot
plt.title(f"{name_str}", fontsize=16)
plt.xlabel("Infection Probability (α)", fontsize=12)
plt.ylabel("Kendall's τ", fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(title="Method", fontsize=10)
plt.tight_layout()

# Place the legend in the upper right corner
plt.legend(title="Method", fontsize=10, loc="upper right")

# Save and show the plot
plt.savefig(f"/content/drive/MyDrive/Complex Networks Project/kendall_tau_comparison_{name_str}.png", dpi=300, bbox_inches = "tight")
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

# Load the uploaded images
img1 = Image.open("/content/drive/MyDrive/Complex Networks Project/kendall_tau_comparison_Karate.png")
img2 = Image.open("/content/drive/MyDrive/Complex Networks Project/kendall_tau_comparison_Jazz.png")
img3 = Image.open("/content/drive/MyDrive/Complex Networks Project/kendall_tau_comparison_Polbooks.png")

# Create a figure with subplots
fig, axs = plt.subplots(1, 3, figsize=(15, 5))  # 1 row, 3 columns

# Display each image in a subplot
axs[0].imshow(img1)
axs[0].axis("off")  # Turn off axes

axs[1].imshow(img2)
axs[1].axis("off")


axs[2].imshow(img3)
axs[2].axis("off")


# Adjust layout for spacing
plt.tight_layout()

# Save the combined figure as a single image
plt.savefig("/content/drive/MyDrive/Complex Networks Project/Comparion_Tau_Networks.png", dpi=300, bbox_inches="tight")

# Show the figure
plt.show()
